### Introduction to Hull White Model

The Hull–White (One-Factor) Model: It is an arbitrage free model.\
dr = (Theta(t) - k * r) * dt + Sigma * dz \
where k and Sigma are constants.\
k is the rate of mean reversion\
Sigma is the long run volatility of short rate. \
Theta(t) is long term mean rate. \
k is determined using log likelihood

In [2]:
import numpy as np
import pandas as pd
import QuantLib as ql
from scipy.optimize import minimize 

In [6]:
#Reading a file containing MIBOR 1 Month rate (proxy for short rate) for last 5 years
RateFile=pd.read_csv("MIBOR1M.csv")

#Keeping only the rate column
RateFile = RateFile[['Mibor1m']]

#Change in daily rate (returns of rate) = dr
returns = RateFile.diff().rename(columns = {'Mibor1m':'dr'})
RateFile = RateFile.join(returns)
RateFile[np.isnan(RateFile)]  = 0

#X is used for determing Log Likelihood
RateFile['X'] = 0
RateFile['LL'] = 0

In [7]:
#initial dummy values for parameter k for mean reversion
k = 0.5

#Theta is the long run mean
Theta = np.average(RateFile[['Mibor1m']])

#Sigma is the long run volatility
Sigma = np.std(RateFile[['Mibor1m']])
Sigma = np.float64(Sigma)

#value of dt is 1 day
dt = 1/252

In [10]:
def HW_Parameter_Est(m, RateFile):
    k = m
    theta = Theta
    sigma = Sigma
    dt = 1/252
    
    #The assumption is [dr] follows Normal Distribution with Mu of [(Theta(t) - k * r) * dt] and Variance of [Sigma**2 * dt]
    #Therefore [(dr - Mu)/sqrt(dt)] follows a Normal Distribution of mu of 0 and Variance of Sigma**2
    #PDF for this function is (1/Sqrt(2*pi*Variance))*exp(-0.5*x**2/Variance)
    #As PDFs are small values where optimization fails at times, the log of PDF is taken for optimization
    #Log of PDF will be log(1/Sqrt(2*pi))+(-0.5)log(Variance)+(-0.5*x**2/Variance)
    #Log(1/Sqrt(2*pi)) is ignored as this is a constant, along with 0.5 of the other two items
    
    for i in range(1,RateFile.shape[0]):
    
        x = RateFile.loc[i,['dr']]
        x = np.float64(x)
    
        y=(theta - k * RateFile.loc[i-1,['Mibor1m']])*dt
        y = np.float64(y)
    
        z=(x-y)/dt**0.5
        z = np.float64(z)
    
        RateFile.loc[i,['X']] = z
    
    for i in range(1,RateFile.shape[0]):
    
        x = np.log(sigma**2)
        x = np.float64(x)
    
        y = RateFile.loc[i,['X']]**2/sigma**2
        y = np.float64(y)
    
        z=-(x+y)
        z = np.float64(z)
    
        RateFile.loc[i,['LL']] = z
        
    Sum_of_LL = np.sum(RateFile['LL'])
    return(-Sum_of_LL)
    

In [12]:
#Optimization Function 
check = minimize(HW_Parameter_Est,x0=k,args=(RateFile))
#Estimated Value of mean Reversion
check.x

array([0.94390871])